### 填充和步幅

##### 填充

#### 填充（padding）是指在输入高和宽的两侧填充元素（通常是0元素）。

In [1]:
from mxnet import nd
from mxnet.gluon import nn

In [3]:
def comp_conv2d(conv2d, X):
    """此函数用来计算卷积层。它初始化卷积层权重，并对输入和输出做相应的升维和降维。"""
    conv2d.initialize()
    X = X.reshape((1, 1)+X.shape)  # (1, 1)代表批量大小和通道数均为1
    Y = conv2d(X)
    return Y.reshape(Y.shape[2:])  # 排除不关心的前两维：批量和通道

In [5]:
conv2d = nn.Conv2D(1, kernel_size=3, padding=1)  # 这里表示两侧分别填充1行或列，所以在两侧一共填充2行或列
X = nd.random.uniform(shape=(8, 8))
comp_conv2d(conv2d, X).shape

(8, 8)

#### 当卷积核的高和宽不同时，可以通过设置高和宽上不同的填充数使输入和输出具有相同的高和宽。

In [7]:
# 使用高和宽分别为5和3的卷积核。在高和宽两侧的填充数分别为2和1
conv2d = nn.Conv2D(1, kernel_size=(5, 3), padding=(2, 1))
comp_conv2d(conv2d, X).shape

(8, 8)

##### 步幅

#### 卷积窗口从输入数组的左上角开始，按从左向右、从上向下的顺序，以此在输入数组上滑动。我们将每次滑动的行数和列数称为步幅（stride）。

In [9]:
conv2d = nn.Conv2D(1, kernel_size=3, strides=2, padding=1)
comp_conv2d(conv2d, X).shape

(4, 4)

In [10]:
conv2d = nn.Conv2D(1, kernel_size=(3, 5), strides=(3, 4), padding=(0, 1))
comp_conv2d(conv2d, X).shape

(2, 2)

### 多输入通道和多输出通道

#### 前面所介绍的输入和输出都是二维数组，但真是的数据维度往往更高。例如，彩色图像在高和宽2个维度之外还有RGB3个颜色通道。

###### 多输入通道

#### 当输入数据含有多个通道时，需要构造一个输入通道数与输入数据的通道数相同的卷积核，从而能够与含多通道的输入数据做互相关运算。

In [11]:
import d2lzh as d2l
from mxnet import nd

In [14]:
def corr2d_multi_in(X, K):
    # 首先沿着X和K的第0维进行遍历。然后使用*将结果列表转换为位置参数传递给add_n函数来进行相加。
    return nd.add_n(*[d2l.corr2d(x, k) for x, k in zip(X, K)])

In [15]:
X = nd.array([[[0, 1, 2], [3, 4, 5], [6, 7, 8]],
             [[1, 2, 3], [4, 5, 6], [7, 8, 9]]])
K = nd.array([[[0, 1], [2, 3]],
             [[1, 2], [3, 4]]])
conv2d_multi_in(X, K)


[[ 56.  72.]
 [104. 120.]]
<NDArray 2x2 @cpu(0)>

##### 多输出通道

#### 当输入通道有多个时，由于我们对各个通道的结果做了累加，所以不论输入通道数是多少，输出通道数总是1.

In [16]:
def corr2d_multi_in_out(X, K):
    # 对K的第0维进行遍历，每次与输入X做互相关运算。所有的结果使用stack函数合并在一起。
    return nd.stack(*[corr2d_multi_in(X, k) for k in K])

In [17]:
K = nd.stack(K, K+1, K+2)
K.shape

(3, 2, 2, 2)

In [18]:
corr2d_multi_in_out(X, K)


[[[ 56.  72.]
  [104. 120.]]

 [[ 76. 100.]
  [148. 172.]]

 [[ 96. 128.]
  [192. 224.]]]
<NDArray 3x2x2 @cpu(0)>

##### 1x1卷积层

#### 假设将通道维当做特征维，把高和宽两个维度上元素当做数据样本，那么1x1卷积层的作用与全连接层等价。下面使用全连接层中的矩阵乘法来实现1x1卷积。需要在矩阵乘运算前后对数据形状做一些调整。

In [19]:
def corr2d_multi_in_out_1x1(X, K):
    c_in, h, w = X.shape
    c_out = K.shape[0]
    X = X.reshape((c_in, h*w))
    K = K.reshape((c_out, c_in))
    return nd.dot(K, X).reshape((c_out, h, w))

#### 经验证，做1x1卷积时，以上函数与之前实现的互相关运算函数corr2d_multi_in_out等价。

In [20]:
X = nd.random.uniform(shape=(3, 3, 3))
K = nd.random.uniform(shape=(2, 3, 1, 1))

Y1 = corr2d_multi_in_out_1x1(X, K)
Y2 = corr2d_multi_in_out(X, K)

(Y1-Y2).norm().asscalar() < 1e-6

True

#### 在之后的模型中，会看到1x1的卷积层被当做保持高和宽维度形状不变的全连接层使用。于是，可以通过调整网路层之间的通道数来控制模型复杂度。

### 池化层

#### 池化(pooling)层的提出是为了缓解卷积层对位置的过渡敏感性。在实际图像里，我们感兴趣的物体不会总出现在固定位置：即使我们连续拍摄同一个物体也极有可能出现像素位置上的偏移。这可能导致同一个边缘对应的输出可能出现在卷积输出Y的不同位置，进而对后面的模式识别造成不便。

##### 二维最大池化层和平均池化层

#### 设卷积层的输入是X、池化层的输出为Y。无论是X[i, j]和X[i, j+1]值不同，还是X[i, j+1]和X[i, j+2]的值不同，池化层均有输出Y[i, j]=1.也就是说，使用2x2最大池化层时，只要卷积层识别的模式在高和宽上移动不超过一个元素，依然可以将它检测出来。

In [21]:
from mxnet import nd
from mxnet.gluon import nn

In [24]:
def pool2d(X, pool_size, mode='max'):
    p_h, p_w = pool_size
    Y = nd.zeros((X.shape[0]-p_h+1, X.shape[1]-p_w+1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            if mode=='max':
                Y[i, j] = (X[i:i+p_h, j:j+p_w]).max()
            elif mode=='avg':
                Y[i, j] = (X[i:i+p_h, j:j+p_w]).mean()
    return Y

In [25]:
X = nd.array([[0, 1, 2], [3, 4, 5], [6, 7, 8]])
pool2d(X, (2, 2))  # 最大池化


[[4. 5.]
 [7. 8.]]
<NDArray 2x2 @cpu(0)>

In [26]:
pool2d(X, (2, 2), 'avg')  # 平均池化


[[2. 3.]
 [5. 6.]]
<NDArray 2x2 @cpu(0)>

In [27]:
nd.stack(X ,X+1)


[[[0. 1. 2.]
  [3. 4. 5.]
  [6. 7. 8.]]

 [[1. 2. 3.]
  [4. 5. 6.]
  [7. 8. 9.]]]
<NDArray 2x3x3 @cpu(0)>

In [29]:
nd.concat(X, X+1, dim=1)


[[0. 1. 2. 1. 2. 3.]
 [3. 4. 5. 4. 5. 6.]
 [6. 7. 8. 7. 8. 9.]]
<NDArray 3x6 @cpu(0)>

##### 填充和步幅

In [31]:
X = nd.arange(16).reshape((1, 1, 4, 4))
X


[[[[ 0.  1.  2.  3.]
   [ 4.  5.  6.  7.]
   [ 8.  9. 10. 11.]
   [12. 13. 14. 15.]]]]
<NDArray 1x1x4x4 @cpu(0)>

#### 默认情况下，MaxPool2D实例里步幅和池化窗口的形状相同。下面使用形状为(3,3)的池化窗口，默认获得形状为(3,3)的步幅。

In [33]:
pool2d = nn.MaxPool2D(3)
pool2d(X)  # 因为池化层没有模型参数，所以不需要调用参数初始化函数


[[[[10.]]]]
<NDArray 1x1x1x1 @cpu(0)>

#### 手动指定步幅和填充

In [34]:
pool2d = nn.MaxPool2D(3, strides=2, padding=1)
pool2d(X)


[[[[ 5.  7.]
   [13. 15.]]]]
<NDArray 1x1x2x2 @cpu(0)>

#### 指定非正方形的池化窗口，并分别指定高和宽上的填充和步幅。

In [35]:
pool2d = nn.MaxPool2D((2, 3), strides=(2, 3), padding=(1, 2))
pool2d(X)


[[[[ 0.  3.]
   [ 8. 11.]
   [12. 15.]]]]
<NDArray 1x1x3x2 @cpu(0)>

##### 多通道

#### 在处理多通道输入数据时，池化层对每个输入通道分别池化，而不是像卷积层那样将各通道的输入按通道相加。这意味着池化层的输出通道数与输入通道数相等。

In [36]:
X = nd.concat(X, X+1, dim=1)
X


[[[[ 0.  1.  2.  3.]
   [ 4.  5.  6.  7.]
   [ 8.  9. 10. 11.]
   [12. 13. 14. 15.]]

  [[ 1.  2.  3.  4.]
   [ 5.  6.  7.  8.]
   [ 9. 10. 11. 12.]
   [13. 14. 15. 16.]]]]
<NDArray 1x2x4x4 @cpu(0)>

In [37]:
pool2d = nn.MaxPool2D(3, strides=2, padding=1)
pool2d(X)


[[[[ 5.  7.]
   [13. 15.]]

  [[ 6.  8.]
   [14. 16.]]]]
<NDArray 1x2x2x2 @cpu(0)>